In [1]:
import bs4 #beautifulsoup 4
from urllib.request import urlopen as uReq #url open module
from bs4 import BeautifulSoup as soup #beautifulsoup module
import datetime
import pandas as pd
import csv
import os
import string

In [2]:
#Hämta en sida
url = "https://store.steampowered.com/search/?specials=1&page=1"
uClient = uReq(url)
page_html = uClient.read()
uClient.close()

In [3]:
#Visa innehållet av datan i spelet
page_soup = soup(page_html, "html.parser")
containers = page_soup.findAll("div",{"class":"responsive_search_name_combined"})
print(len(containers))
print(containers[0])

25
<div class="responsive_search_name_combined">
<div class="col search_name ellipsis">
<span class="title">Atelier Arland series Deluxe Pack - アトリエ ～アーランドの錬金術士１・２・３～ DX</span>
<p>
<span class="platform_img win"></span> </p>
</div>
<div class="col search_released responsive_secondrow"></div>
<div class="col search_reviewscore responsive_secondrow">
<span class="search_review_summary positive" data-tooltip-html="Positive&lt;br&gt;100% of the 26 user reviews for games in this bundle are positive.">
</span>
</div>
<div class="col search_price_discount_combined responsive_secondrow">
<div class="col search_discount responsive_secondrow">
<span>-33%</span>
</div>
<div class="col search_price discounted responsive_secondrow">
<span style="color: #888888;"><strike>119,97€</strike></span><br/>80,97€							</div>
</div>
</div>


In [4]:
#Ta in positiva ratings and antalet reviews, om de finns
try:
    rating = containers[0].find("span", {"class","search_review_summary"})
    ratingtext = rating["data-tooltip-html"]
    splitrating = ratingtext.split(">")
    splitrating[1].split()
    percentile = splitrating[1].split()
    score = percentile[0]
    reviews = percentile[3]
    print(score, reviews)
except:
    print("No review data")

100% 26


In [5]:
#Ta in rabatten
discount = containers[0].find("div", {"class","search_discount"})
print(discount.span.text)

-33%


In [6]:
#Hämta priserna
price = containers[0].find("div", {"class","search_price"}).text
price2 = price.split("€")
originalprice = price2[0].replace("\n","")
discountprice = price2[1]
print(originalprice, discountprice)

119,97 80,97


In [7]:
#Hämta datum
published = containers[1].find("div", {"class","search_released"})
print(published.text)

13 Oct, 2016


In [8]:
#Stöder windows?
platformwin = containers[0].find("span", {"class","platform_img win"})
try:
    if len(platformwin) == 0:
        supportwin = 1
except TypeError:
    supportwin = 0
print(supportwin)

1


In [9]:
#Stöder mac?
platformmac = containers[0].find("span", {"class","platform_img mac"})
try:
    if len(platformmac) == 0:
        supportmac = 1
except TypeError:
    supportmac = 0
print(supportmac)

0


In [10]:
#Stöder linux?
platformlinux = containers[0].find("span", {"class","platform_img linux"})
try:
    if len(platformlinux) == 0:
        supportlinux = 1
except TypeError:
    supportlinux = 0
print(supportlinux)

0


In [11]:
#Dagens datum
now = datetime.datetime.now()
print (now.strftime("%Y-%m-%d %H:%M"))

2018-12-04 18:48


In [12]:
#Hämta titeln, konvertera till byts och tillbaka till sträng (måste göras p.g.a. kinesiska tecken, annars går det inte att skriva i csv)
title = containers[0].find("span", {"class","title"}).text
titlebyte = title.encode('ascii', 'ignore')
str(titlebyte, 'utf-8')

'Atelier Arland series Deluxe Pack -   DX'

In [13]:
#Skapa csv fil med headers
filename="steamsales.csv"
file = open(filename, 'a')
writer = csv.writer(file)
if os.stat("steamsales.csv").st_size == 0:
    writer.writerow(["Title", "Rating", "reviews", "Discount", "Normal price", "discount price", "Publish date", "Win", "Mac", "Linux", "Date"])

In [14]:
#De tidigare kodsnuttarna lagt ihop till en loop
class Scraper:
    def scrape(self):
        for x in range(len(containers)):
            title = containers[x].find("span", {"class","title"}).text
            titlebyte = title.encode('ascii', 'ignore')
            title=str(titlebyte, 'utf-8')
            print(title)

            try:
                rating = containers[x].find("span", {"class","search_review_summary"})
                ratingtext = rating["data-tooltip-html"]
                splitrating = ratingtext.split(">")
                splitrating[1].split()
                percentile = splitrating[1].split()
                score = percentile[0]
                reviews = percentile[3].strip()
                print(score)
                print(reviews)
            except:
                print("No review data")
            discount = containers[x].find("div", {"class","search_discount"})
            price = containers[x].find("div", {"class","search_price"}).text
            price2 = price.split("€")
            originalprice = price2[0].replace("\n","").strip()
            discountprice = price2[1].strip()
            published = containers[x].find("div", {"class","search_released"}).text
            discounted = discount.text.strip()
            print(discounted)
            print(originalprice)
            print(discountprice)
            print(published)
            platformwin = containers[x].find("span", {"class","platform_img win"})
            try:
                if len(platformwin) == 0:
                    supportwin = 1
            except TypeError:
                supportwin = 0
            print("Windows support:",supportwin)

            platformmac = containers[x].find("span", {"class","platform_img mac"})
            try:
                if len(platformmac) == 0:
                    supportmac = 1
            except TypeError:
                supportmac = 0
            print("Mac support:",supportmac)

            platformlinux = containers[x].find("span", {"class","platform_img linux"})
            try:
                if len(platformlinux) == 0:
                    supportlinux = 1
            except TypeError:
                supportlinux = 0
            print("Linux support:",supportlinux)
            now = datetime.datetime.now()
            print (now.strftime("%Y-%m-%d %H:%M"))
            saledate= now.strftime("%Y-%m-%d %H:%M")
            writer.writerow([title,score,reviews,discounted,originalprice,discountprice, published,supportwin,supportmac,supportlinux,saledate])
scraper = Scraper()
scraper.scrape()

Atelier Arland series Deluxe Pack -   DX
100%
26
-33%
119,97
80,97

Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Shadow Warrior 2
87%
8,895
-66%
36,99
12,57
13 Oct, 2016
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Tom Clancys The Division
67%
53,141
-75%
49,99
12,49
7 Mar, 2016
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Valley
90%
3,287
-90%
19,99
1,99
24 Aug, 2016
Windows support: 1
Mac support: 1
Linux support: 1
2018-12-04 18:48
The Elder Scrolls Online
76%
33,017
-50%
19,99
9,99
22 May, 2017
Windows support: 1
Mac support: 1
Linux support: 0
2018-12-04 18:48
Insurgency: Sandstorm
No review data
-10%
29,99
26,99
12 Dec, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Pillars of Eternity - Deadfire Pack
95%
74
-50%
36,99
Free
15 Nov, 2017
Windows support: 1
Mac support: 1
Linux support: 1
2018-12-04 18:48
Pillars of Eternity - The White March Part II
84%
76
-40%
14,99
8,99
16 Feb, 2016
Wi

In [15]:
#Hämta 5 sidor och lagra datan i en csv
for x in range(5):
    page=str(x+1)
    url = ("https://store.steampowered.com/search/?specials=1&page="+page)
    uClient = uReq(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, "html.parser")
    containers = page_soup.findAll("div",{"class":"responsive_search_name_combined"})
    scraper.scrape()

Atelier Arland series Deluxe Pack -   DX
100%
26
-33%
119,97
80,97

Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Shadow Warrior 2
87%
8,895
-66%
36,99
12,57
13 Oct, 2016
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Tom Clancys The Division
67%
53,141
-75%
49,99
12,49
7 Mar, 2016
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Valley
90%
3,287
-90%
19,99
1,99
24 Aug, 2016
Windows support: 1
Mac support: 1
Linux support: 1
2018-12-04 18:48
The Elder Scrolls Online
76%
33,017
-50%
19,99
9,99
22 May, 2017
Windows support: 1
Mac support: 1
Linux support: 0
2018-12-04 18:48
Insurgency: Sandstorm
No review data
-10%
29,99
26,99
12 Dec, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Pillars of Eternity - Deadfire Pack
95%
74
-50%
36,99
Free
15 Nov, 2017
Windows support: 1
Mac support: 1
Linux support: 1
2018-12-04 18:48
Pillars of Eternity - The White March Part II
84%
76
-40%
14,99
8,99
16 Feb, 2016
Wi

You're Fired
51%
31
-90%
3,99
0,39
11 May, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Caligo
76%
380
-90%
5,99
0,59
22 Sep, 2017
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Press X to Not Die
91%
1,877
-80%
2,99
0,59
6 Oct, 2017
Windows support: 1
Mac support: 1
Linux support: 1
2018-12-04 18:48
Fitzzle Wise Owls
No review data
-80%
2,39
0,47
6 Jul, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Stickman.io
No review data
-88%
3,29
0,39
28 May, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Trivia Vault: Mixed Trivia 2
78%
14
-90%
4,99
0,49
13 Feb, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Lost Artifacts - Ancient Tribe Survival
78%
19
-90%
4,99
0,49
1 Feb, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:48
Superfighters Deluxe
93%
136
-10%
10,79
9,71
30 Nov, 2018
Windows support: 1
Mac support: 0
Linux support: 0
2018-12-04 18:4

In [16]:
file.close()